In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import optuna
from sklearn.model_selection import KFold, cross_validate


In [2]:
df = pd.read_csv("gender_stats_preprocessed.csv")
df = df.dropna(axis=1)
x = df.drop(columns=['Country Name'], axis=0)
x = x.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled, columns=df.columns[1:], index=df['Country Name'])

In [3]:
train_data, test_data = train_test_split(df_scaled, random_state=43)

In [4]:
if "Poland" not in test_data.index:
    test_data.loc['Poland'] = train_data.loc['Poland']
    train_data.drop('Poland', inplace=True)

In [5]:
y_train = train_data['Fertility rate, total (births per woman)'].copy()
X_train = train_data.drop('Fertility rate, total (births per woman)', axis=1)

y_test = test_data['Fertility rate, total (births per woman)'].copy()
X_test = test_data.drop('Fertility rate, total (births per woman)', axis=1)

In [14]:
def objective(trial):
    # Suggest values for the hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    units_first_layer = trial.suggest_int("units_first_layer", 10, 100)
    units_second_layer = trial.suggest_int("units_second_layer", 10, 100)
    activation = trial.suggest_categorical("activation", ["relu", "selu", "elu", "swish"])

    # Build the Keras model (as per your provided structure)
    model = keras.Sequential([
        layers.Dense(units_first_layer, activation),
        layers.Dense(units_second_layer, activation),
        layers.Dense(1)
    ])

    # Compile the model
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate))

    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=0)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test, verbose=0)
    return loss


In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2024-01-21 14:28:28,613] A new study created in memory with name: no-name-d45157c7-bb0b-4016-8451-21e9b54197d5
[I 2024-01-21 14:28:29,235] Trial 0 finished with value: 0.1570568084716797 and parameters: {'learning_rate': 0.0024363924239892668, 'units_first_layer': 100, 'units_second_layer': 23, 'activation': 'swish'}. Best is trial 0 with value: 0.1570568084716797.
[I 2024-01-21 14:28:29,588] Trial 1 finished with value: 0.20315992832183838 and parameters: {'learning_rate': 0.001149381615465511, 'units_first_layer': 21, 'units_second_layer': 62, 'activation': 'selu'}. Best is trial 0 with value: 0.1570568084716797.
[I 2024-01-21 14:28:29,941] Trial 2 finished with value: 0.39364394545555115 and parameters: {'learning_rate': 0.00852026753036606, 'units_first_layer': 12, 'units_second_layer': 29, 'activation': 'selu'}. Best is trial 0 with value: 0.1570568084716797.
[I 2024-01-21 14:28:30,325] Trial 3 finished with value: 0.5535844564437866 and parameters: {'learning_rate': 1.44298398

In [13]:
print("Best trial:")
trial = study.best_trial

print(" Value of loss:", trial.value)
print(" Best hyperparameters:", trial.params)

Best trial:
 Value of loss: 0.1298944354057312
 Best hyperparameters: {'learning_rate': 0.003655316713200898, 'units_first_layer': 11, 'units_second_layer': 71, 'activation': 'relu'}


In [15]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2024-01-21 14:30:54,487] A new study created in memory with name: no-name-1ed4db4e-fb36-4abe-8a62-98d25b500370
[I 2024-01-21 14:30:54,799] Trial 0 finished with value: 0.19664347171783447 and parameters: {'learning_rate': 0.0037614400386799607, 'units_first_layer': 53, 'units_second_layer': 51, 'activation': 'relu'}. Best is trial 0 with value: 0.19664347171783447.
[I 2024-01-21 14:30:55,096] Trial 1 finished with value: 1.001639485359192 and parameters: {'learning_rate': 0.00010417996041379952, 'units_first_layer': 63, 'units_second_layer': 65, 'activation': 'elu'}. Best is trial 0 with value: 0.19664347171783447.
[I 2024-01-21 14:30:55,384] Trial 2 finished with value: 0.2785108983516693 and parameters: {'learning_rate': 0.006262402126916757, 'units_first_layer': 38, 'units_second_layer': 50, 'activation': 'relu'}. Best is trial 0 with value: 0.19664347171783447.
[I 2024-01-21 14:30:55,693] Trial 3 finished with value: 0.18453969061374664 and parameters: {'learning_rate': 0.006671

In [16]:
print("Best trial:")
trial = study.best_trial

print(" Value of loss:", trial.value)
print(" Best hyperparameters:", trial.params)

Best trial:
 Value of loss: 0.13272441923618317
 Best hyperparameters: {'learning_rate': 0.002691225139497515, 'units_first_layer': 25, 'units_second_layer': 29, 'activation': 'swish'}
